## Target number of units

In [ ]:
from sqlalchemy import create_engine
from pysandag.database import get_connection_string
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib inline

In [ ]:
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

In [ ]:
capacity_sql = '''
SELECT sum([cap_remaining_new]) as capacity
  FROM [urbansim].[urbansim].[parcel_update_2017]'''
capacity_df =  pd.read_sql(capacity_sql, mssql_engine)
capacity = int(capacity_df.values)

In [ ]:
units_needed_sql = '''
SELECT sum(housing_units_add) as housing_units_to_add
FROM [isam].[economic_output].[urbansim_housing_units]'''
units_needed_df =  pd.read_sql(units_needed_sql, mssql_engine)
units_needed = int(units_needed_df.values)

In [ ]:
housing_unit_sql = '''select  yr,
    housing_units1, households,housing_units_add 
    from [isam].[economic_output].[urbansim_housing_units]'''
hu_df =  pd.read_sql(housing_unit_sql, mssql_engine)
hu_df['total_housing_units_add'] = hu_df.housing_units_add.cumsum()

In [ ]:
existing_du_sql = '''
  select sum(du_2017) 
  from [urbansim].[urbansim].[parcel_update_2017]'''
existing_du_df =  pd.read_sql(existing_du_sql, mssql_engine)
existing_du = int(existing_du_df.values)

# Target

In [ ]:
print("Target units to add: {:,}".format(units_needed))

# Capacity

In [ ]:
print("Capacity: {:,}".format(capacity))

# Remaining

In [ ]:
print("Remaining units: {:,}".format(capacity - units_needed))

# Target units 2017

In [ ]:
units_to_add_2017 = int(hu_df.loc[hu_df.yr == 2017].housing_units_add.values[0])
print("Target units to add 2017: {:,}".format(units_to_add_2017))

# Existing du 2017 urbansim.parcel_update_2017

In [ ]:
print("Existing du 2017 from urbansim.parcel_update_2017: {:,}".format(existing_du))

# Calculate number of housing units: existing du + target units

In [ ]:
housing_units_end_of_2017 = units_to_add_2017 + existing_du
print("Housing units at end of 2017: {:,}".format(housing_units_end_of_2017))

# Database number of housing units [isam].[economic_output].[urbansim_housing_units]

In [ ]:
hu_df_2017 = int(hu_df.loc[hu_df.yr == 2017].housing_units1.values[0])
print("Housing units at end of 2017 from database table: {:,}".format(hu_df_2017))

#  <span style="color:red">Difference between database table and calculated units </span>

In [ ]:
diff_hu_2017 = hu_df_2017 - housing_units_end_of_2017
print("Difference between database and calculation for 2017 housing units: {:,}".format(diff_hu_2017))

# Plot (note: increase 2026 and 2027)

In [ ]:
existing_du

In [ ]:
hu_df['capacity'] = capacity

In [ ]:
hu_df['remaining_capacity'] = hu_df['capacity'] - hu_df['total_housing_units_add'] 

In [ ]:
# hu_df

In [ ]:
fig, axes = plt.subplots(3, 1)


plot_df = hu_df[['yr','housing_units_add']].copy()
plot_df.rename(columns = {'housing_units_add': 'Housing units to add'}, inplace=True)
plot_df.set_index('yr',inplace=True)
plot_df.plot(style='.-',ax=axes.flat[0],figsize=(10,5),color='b')

plot_df = hu_df[['yr','total_housing_units_add','households']].copy()
plot_df.set_index('yr',inplace=True)
plot_df['Total Housing Units'] = existing_du + plot_df['total_housing_units_add']
plot_df.rename(columns = {'households': 'Households'}, inplace=True)
del  plot_df['total_housing_units_add']
plot_df.plot(style='.-',ax=axes.flat[1],figsize=(10,10),color=['darkorange','b'])

plot_df = hu_df[['yr','total_housing_units_add','remaining_capacity']].copy()
plot_df.set_index('yr',inplace=True)
plot_df.rename(columns = {'total_housing_units_add': 'Housing units cusum'}, inplace=True)
plot_df.rename(columns = {'remaining_capacity': 'Remaining capacity'}, inplace=True)
plot_df.plot(style='.-',ax=axes.flat[2],figsize=(10,15),color=['b','chocolate'])
plt.axhline(y=0,color='black', lw=2, alpha=0.5,linestyle='--')
# format
fmt = '{x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)

axes.flat[0].set_xlabel("Simulation Year",size =16)
axes.flat[0].set_ylabel("Housing Units",size =16)
axes.flat[0].set_title('Housing Units Needed by Year\n demographic_simulation_id=2001',size=16) 
axes.flat[0].yaxis.set_major_formatter(tick) 
axes.flat[0].legend(fontsize=14)

# change order of legend in second plot
axes.flat[1].set_xlabel("Simulation Year",size=16)
axes.flat[1].set_ylabel("Total Housing Units",size =16)
axes.flat[1].set_title('Housing Units and Households \ndemographic_simulation_id=2001',size=16) 
axes.flat[1].yaxis.set_major_formatter(tick) 
handles, labels = axes.flat[1].get_legend_handles_labels()
zipped = zip(labels,handles)
zipped.sort(key = lambda t: t[0],reverse=True)
labels, handles = zip(*zipped)
axes.flat[1].legend(handles, labels,fontsize=14)
# axes.flat[1].legend(fontsize=14)

axes.flat[2].set_xlabel("Simulation Year",size =16)
axes.flat[2].set_ylabel("Housing Units (cusum)",size =16)
axes.flat[2].set_title('Housing Units Needed and Remaining Capacity\n demographic_simulation_id=2001',size=16) 
axes.flat[2].yaxis.set_major_formatter(tick) 
axes.flat[2].legend(fontsize=14)

plt.xticks(rotation=25)
plt.tight_layout(pad=0.5, w_pad=0.6, h_pad=2)



plt.show()
fig.savefig('Demographic_simulation_2001.png', format='png', dpi=300)

# Previous Version:

### Capacity

In [ ]:
parcels_capacity_sql = '''
SELECT  SUM(capacity) AS capacity_parcels
FROM urbansim.urbansim.parcel where capacity > 0
'''
parcel_capacity_df =  pd.read_sql(parcels_capacity_sql, mssql_engine)
parcel_capacity = int(parcel_capacity_df.values)

In [ ]:
print("Original capacity: {:,}".format(parcel_capacity))

In [ ]:
print("Parcel update 2017 capacity: {:,}".format(capacity))

In [ ]:
print("Capacity difference: update 2017 minus original capacity from parcel table: {:,}".
      format(capacity-parcel_capacity))

### Dwelling units

In [ ]:
buildings_sql = '''
SELECT  SUM(COALESCE(residential_units,0)) AS residential_units
FROM urbansim.urbansim.building
'''

In [ ]:
du_df =  pd.read_sql(buildings_sql, mssql_engine)

In [ ]:
du = int(du_df.values)

In [ ]:
print("Existing dwelling units: {:,}".format(du))

### Households 2050

In [ ]:
household_sql = '''
  SELECT sum(hh) AS hh
  FROM isam.demographic_output.summary
  WHERE sim_id = 1004 and yr = 2050
  '''

In [ ]:
hh_df =  pd.read_sql(household_sql, mssql_engine)

In [ ]:
hh = int(hh_df.values)

In [ ]:
print("Number of households 2050: {:,}".format(hh))

### Target number of units for region

In [ ]:
target_for_region = hh - du

In [ ]:
print("Target for region: {:,}".format(target_for_region))

# Map with folium

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[32.7157,  -117.1611])

In [ ]:
m